Rick Astley 1987年發表的<<Never Gonna Give You Up>>在那個年代紅遍世界，2007年因為網路的興起，以及釣魚網站，從而讓rickroll變成了迷因(MEME)。而到了2018年，因為電影<<無敵破壞王2>>片尾對rickroll的致敬，又再次讓rickroll掀起熱潮。到了2022年，又再度以業配的形式重新唱了一遍，經歷了將近35年，歌聲卻幾乎沒變。
製作這個影片純粹是對rick致敬，以及，沒錯，你被roll了

In [1]:
import cv2
import numpy as np
import dlib

# 眼睛追蹤
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../model/shape_predictor_68_face_landmarks.dat')


# 辨識正面人臉
face_cascade = cv2.CascadeClassifier(r'../model/haarcascade_frontalface_default.xml')
# 辨識眼睛
eye_cascade = cv2.CascadeClassifier('../model/haarcascade_eye.xml')

# 解碼器為 mp4v
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# 原始影片
cap = cv2.VideoCapture('movie.mp4')

# 指定輸出影片的寬度、高度以及幀速率
width = int(cap.get(3))
height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 輸出的影片檔名為output.mp4，以mp4v編碼，其餘皆按照上面輸出
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

# 設定旋轉角度和中心點
angle = 0
center = (width // 2, height // 2)

# 開始以幀數切割影片
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        # 獲取當前幀數
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
        
        # 不同的幀數，不同的效果
        if current_frame < 200:
            # 加上字串
            cv2.putText(frame, "split bgr then merge rgb", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            b, g, r = cv2.split(frame)
            merged = cv2.merge([r, g, b])
            frame = cv2.addWeighted(frame, 0.5, merged, 0.5, 0)
            # 將處理完的幀寫入輸出影片中
            out.write(frame)
        elif current_frame < 400:
            # 加上字串
            cv2.putText(frame, "Edge detection", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # 在200到400幀之間，將圖像進行邊緣檢測
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            edges = cv2.Canny(blurred, 50, 150)
            frame = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

            # 將處理完的幀寫入輸出影片中
            out.write(frame)
        elif current_frame < 600:
            # 加上字串
            cv2.putText(frame, "gray", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # 再400~600幀之間，將圖像轉為灰色
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
            # 將處理完的幀寫入輸出影片中
            out.write(frame)
        elif current_frame < 800:
            # 加上字串
            cv2.putText(frame, "rotate", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # 旋轉影像
            M = cv2.getRotationMatrix2D(center, angle, 1)
            frame = cv2.warpAffine(frame, M, (width, height))
            # 增加角度
            angle += 1.77
            if angle >= 360:
                angle = 0
            # 將旋轉後的影像寫入輸出影片中
            out.write(frame)
        elif current_frame < 1000:
            # 加上字串
            cv2.putText(frame, "face detection", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # 檢測影片中的人臉
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            # 繪製人臉識別框框
            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            out.write(frame)
        elif current_frame < 1200:
            # 加上字串
            cv2.putText(frame, "eyes detection cascade", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # 將影像轉換為灰度圖像
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # 檢測眼睛
            eyes = eye_cascade.detectMultiScale(gray)

            # 繪製圓圈
            for (x, y, w, h) in eyes:
                cx, cy = x + w // 2, y + h // 2
                r = int((w + h) * 0.25)
                cv2.circle(frame, (cx, cy), r, (0, 255, 0), thickness=2)
            out.write(frame)
        elif current_frame < 1400:
            # 加上字串
            cv2.putText(frame, "shape_predictor_68_face_landmarks", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            # 檢測人臉
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)

            # 追蹤眼睛
            for face in faces:
                landmarks = predictor(gray, face)
                left_eye_points = []
                right_eye_points = []
            
                # 擷取左眼和右眼的特徵點
                for n in range(36, 42):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y
                    left_eye_points.append((x, y))
                for n in range(42, 48):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y
                    right_eye_points.append((x, y))
            
                # 繪製眼睛輪廓
                cv2.polylines(frame, [np.array(left_eye_points)], True, (0, 255, 0), 2)
                cv2.polylines(frame, [np.array(right_eye_points)], True, (0, 255, 0), 2)

                # 計算左眼和右眼的中心位置
                left_eye_center = np.mean(left_eye_points, axis=0).astype(int)
                right_eye_center = np.mean(right_eye_points, axis=0).astype(int)

                # 繪製眼睛中心位置
                cv2.circle(frame, tuple(left_eye_center), 2, (0, 0, 255), -1)
                cv2.circle(frame, tuple(right_eye_center), 2, (0, 0, 255), -1)
            # 將旋轉後的影像寫入輸出影片中
            out.write(frame)
        else:
            # 把原始影像寫入輸出影片中
            out.write(frame)

        cv2.imshow('frame', frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

